# Monitoreo de Modelos de ML con Application Insights

¿Qué es Application Insights? Servicio de administración del rendimiento de aplicaciones de Azure.

- Permite capturar, almacenar y analizar datos de telemetría de aplicaciones (independientemente de si se ejecutan en Azure).
- Requiere un paquete de instrumentación ligero para capturar y enviar datos.

Integración con Azure ML
- El paquete necesario ya está incluido en los servicios web de Azure ML.
- Permite capturar y revisar datos de telemetría para modelos publicados con Azure Machine Learning.

#### Habilitar Application Insights para Monitoreo de Modelos

- Recurso de Application Insights asociado a su espacio de trabajo de Azure ML.
- Servicio configurado para usar ese recurso para registro de telemetría.

Asociación de Application Insights
- Se selecciona un recurso existente al crear el espacio de trabajo de Azure Machine Learning.
- Si no se selecciona ninguno, se crea uno automáticamente.
- Se puede verificar el recurso asociado en la página "Resumen" del espacio de trabajo en el portal de Azure.

Habilitación de Application Insights
- Para **Nuevos servicios**: Habilitar durante la configuración de despliegue del servicio en tiempo real.
- Para **Servicios existentes**:
  - Modificar la configuración de despliegue para servicios basados en Azure Kubernetes Service (AKS) en el portal de Azure.
  - Actualizar cualquier servicio web usando el SDK de Azure ML.


In [ ]:
dep_config = AciWebservice.deploy_configuration(cpu_cores=1, 
                                                memory_gb=1, 
                                                enable_app_insights=True)

#### Captura y visualización de telemetría
Application Insights captura automáticamente toda la información escrita en los registros de errores y resultados estándar, y proporciona una funcionalidad de consulta para ver los datos de estos registros.

##### Escribir datos de registro
Para capturar datos de telemetría para Application Insights, puede escribir cualquier valor en el registro de salida estándar en el script de puntuación del servicio mediante una instrucción **print**, como se muestra en el ejemplo siguiente:

In [ ]:
def init():
    global model
    model = joblib.load(Model.get_model_path('my_model'))
def run(raw_data):
    data = json.loads(raw_data)['data']
    predictions = model.predict(data)
    log_txt = 'Data:' + str(data) + ' - Predictions:' + str(predictions)
    print(log_txt)
    return predictions.tolist()

Azure Machine Learning crea una dimensión personalizada en el modelo de datos de Application Insights para la salida que escriba.

Registros de consultas en Application Insights
Para analizar los datos de registro capturados, puede usar la interfaz de consulta de Log Analytics para Application Insights en Azure Portal. Esta interfaz admite una sintaxis de consulta similar a SQL que puede usar para extraer campos de los datos registrados, incluidas las dimensiones personalizadas creadas por el servicio Azure Machine Learning.

Por ejemplo, la siguiente consulta devuelve los  campos `timestamp` y `customDimensions.Content` de los seguimientos de registro que tienen un  valor de campo de **message** de STDOUT (que indica que los datos están en el registro de salida estándar) y un `customDimensions`. `["Service Name"]` valor de *my-svc*.

In [ ]:
traces
|where message == "STDOUT"
  and customDimensions.["Service Name"] = "my-svc"
| project  timestamp, customDimensions.Content

Documentacion oficial: [Monitor online endpoints](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-monitor-online-endpoints?view=azureml-api-2)